In [4]:
import pandas as pd

df = pd.read_csv('../data/raw/sentipolc/training_set_sentipolc16.csv')

In [10]:
df.head(200)

,idtwitter,subj,opos,oneg,iro,lpos,lneg,top,text
0,122449983151669248,1,0,1,0,0,1,1,Intanto la partita per Via Nazionale si compli...
1,125485104863780865,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà Mario Monti..."
2,125513454315507712,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà #editoriale..."
3,125524238290522113,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...
4,125527933224886272,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...
...,...,...,...,...,...,...,...,...,...
195,134555393362886656,1,0,0,0,0,0,1,"Mario Monti, neo senatore a vita, «SuperMario»..."
196,134556260380053504,1,0,1,0,0,1,1,@LUltimaParola Mario Monti ennesimo regalo all...
197,134556844122324992,0,0,0,0,0,0,1,#sondaggio Governo Monti o elezioni anticipate...
198,134557222494666752,1,0,1,0,0,1,1,"per unire politica e nerdate, Mario Monti è qu..."


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0', num_labels=2)

Downloading: 100%|██████████| 625/625 [00:00<00:00, 183kB/s]
Downloading: 100%|██████████| 1.06M/1.06M [00:00<00:00, 1.77MB/s]
Downloading: 100%|██████████| 706M/706M [01:38<00:00, 7.52MB/s] 
Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a m

In [45]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re

def twitter_preprocess(text:str) -> str:
    text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
    )

    text = str(" ".join(text_processor.pre_process_doc(text)))
    text = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = re.sub(r'(\w)\1{2,}',r'\1\1', text)
    text = re.sub ( r'^\s' , '' , text )
    text = re.sub ( r'\s$' , '' , text )

    return text

df['processed_text'] = df['text'].map(twitter_preprocess)

Word statistics files not found!
Downloading... 

HTTPError: HTTP Error 404: Not Found

In [33]:
train_set = SentipolcSubjDataset(df.head(200), 'm-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0')